# Setting

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import sys
from IPython.display import Image
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings
import tensorflow as tf
import tensorflow_datasets as tfds
warnings.filterwarnings("ignore")

# 15.1 합성곱 신경망의 구성요소

합성곱 신경망 (Convolutional Neural Network)은 뇌의 시각 피질이 물체를 인식할 때 동작하는 방식에서 영감을 얻은 모델

이미지 분류 작업에서 탁월한 성능을 내고 컴퓨터 비전을 위한 머신 러닝 분야를 크게 발전시켰다.


## 15.1.1 CNN과 특성 계층 학습

핵심 특징을 올바르게 추출하는 것은 모든 머신러닝 알고리즘의 성능에서 가장 중요하다.

CNN은 원본 데이터에서 작업에 가장 유용한 특성을 자동으로 학습할 수 있다.

이런 이유로 CNN 층을 특성 추출기로 생각하기도 한다.

각 층별로 저수준 특성을 연결하여 고수준 특성을 만듦으로써 소위 특성 계층을 구성한다.

입력 이미지에서 특성 맵을 만든다.

이 맵의 각 원소는 입력 이미지의 국부적인 픽셀 패치에서 유도된다.

In [3]:
Image(url='https://git.io/JL5O3', width=700)

이런 국부적인 픽셀 패치를 국부 수용장이라고 한다.

CNN은 다음 두 개의 주요 아이디어 덕분에 이미지 관련 작업을 매우 잘 수행한다.

1. 최소 연결 : 특성 맵에 있는 하나의 원소는 하나의 픽셀 패치에만 연결된다.

2. 파라미터 공유 : 동일한 가중치가 입력이미지의 모든 패치에 사용된다.

이 두 아이디어 결과로 일반적인 완전 연결 MLP를 합성곱 층으로 바꾸면 네트워크의 가중치 개수가 극적으로 감소하고 중요 특징을 잡아내는 능력이 향상된다.

이미지 데이터를 보면 가까이 있는 픽셀들이 멀리 떨어져 있는 픽셀보다 연관성이 높다고 가정할 수 있다.

일반적으로 CNN은 여러 개의 합성곱 층과 폴링이라고 하는 서브샘플링 층으로 이루어져 있다.

마지막에는 하나 이상의 완전 연결층이 따라온다.

폴링 층으로 알려진 서브 샘플링 층은 학습되는 파라미터가 없다.

## 15.1.2 이산 합성곱 수행

이산 합성곱이 CNN의 기본 연산이다.

x는 입력으로 보고 w는 필터 혹은 커널이라고 한다.

1차원 이산 합성곱 연산 수행

In [3]:
Image(url='https://git.io/JL5O8', width=700)

지금까지 유한한 크기의 출력 벡터를 얻기 위해 합성곱에 제로 패딩을 사용했다.

패딩을 적용 안 하면 첫 번째 원소는 출력값에 한 번 활용되고 두 번째 원소는 두 번 활용되는 경우가 생길 수 있다.

실전에서 자주 사용하는 패딩은 full padding, same padding, valid padding이다.

In [4]:
Image(url='https://git.io/JL5Ow', width=700)

- 풀패딩은 패딩 파라미터 p를 m-1로 설정한다. 출력 크기를 증가시키기 때문에 합성곱 신경망에서 거의 사용되지 않는다.

- 세임패딩은 출력 크기가 입력 벡터와 같아야 할 때 사용한다.

- 밸리드 패딩은 p=0인 경우를 말한다.

합성곱 출력 크기 계산

In [14]:
def conv1d(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate(
            [zero_pad, x_padded, zero_pad])
    res = []
    for i in range(0, int((len(x_padded) - len(w_rot)) / s) + 1, s):
        res.append(np.sum(
            x_padded[i:i+w_rot.shape[0]] * w_rot))
    return np.array(res)

In [15]:
## 테스트:
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]

print('Conv1d 구현:',
      conv1d(x, w, p=2, s=1))

print('넘파이 결과:',
      np.convolve(x, w, mode='same')) 

Conv1d 구현: [ 5. 14. 16. 26. 24. 34. 19. 22.]
넘파이 결과: [ 5 14 16 26 24 34 19 22]


2D 이산 합성곱 수행

In [17]:
Image(url='https://git.io/JL5OP', width=700)

In [18]:
Image(url='https://git.io/JL5OD', width=600)

In [19]:
Image(url='https://git.io/JL5OS', width=800)

In [30]:
import scipy.signal

def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1,::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2*p[0]
    n2 = X_orig.shape[1] + 2*p[1]
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0]+X_orig.shape[0],
    p[1]:p[1]+X_orig.shape[1]] = X_orig

    res = []
    for i in range(0, int((X_padded.shape[0] - 
                           W_rot.shape[0])/s[0])+1, s[0]):
        res.append([])
        for j in range(0, int((X_padded.shape[1] - 
                               W_rot.shape[1])/s[1])+1, s[1]):
            X_sub = X_padded[i:i+W_rot.shape[0],
                             j:j+W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    return(np.array(res))

In [31]:
X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
W = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]

print('Conv2d 구현:\n',
    conv2d(X, W, p=(1, 1), s=(1, 1)))


print('싸이파이 결과:\n',
    scipy.signal.convolve2d(X, W, mode='same'))

Conv2d 구현:
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]
싸이파이 결과:
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


## 15.1.3 서브샘플링

서브샘플링은 전형적인 두 종류의 풀링 연산으로 합성곱 신경망에 적용된다.

최대 풀링, 평균 풀링.

In [32]:
Image(url='https://git.io/JL5OH', width=700)